In [1]:
from statsmodels.iolib.smpickle import load_pickle
import pandas as pd
import numpy as np

In [2]:
margin_res = load_pickle("models/margin_res.pickle")
total_res = load_pickle("models/total_res.pickle")

In [3]:
team = "GBP"
opponent = "ARI"
third = .452
third_allowed = .324
top = 28.90
a_top = 31.10
first_downs = 23
first_downs_allowed = 19
pass_yards = 345
pass_yards_allowed = 249
penalty_yards = 112
a_penalty_yards = 81
plays = 82
a_plays = 73
rush_yards = 122
rush_yards_allowed = 186
sacked = 4
sacks = 3
takeaways = 2
total_yards = 467
total_yards_allowed = 435
turnovers = 1

home_dictionary = {"ha": "home", "team": team, "opp": opponent, "third_per": third, "third_per_allowed": third_allowed, 
                   "TOP": top, "first_downs": first_downs, "first_downs_allowed": first_downs_allowed, "pass_yards": pass_yards,
                   "pass_yards_allowed": pass_yards_allowed, "penalty_yards": penalty_yards, "plays": plays, 
                   "rush_yards": rush_yards, "rush_yards_allowed": rush_yards_allowed, "sacks": sacks, "sacked": sacked,
                   "takeaways": takeaways, "turnovers": turnovers, "total_yards": total_yards, 
                   "total_yards_allowed": total_yards_allowed}

away_dictionary = {"ha": "away", "team": opponent, "opp": team, "third_per": third_allowed, "third_per_allowed": third, 
                   "TOP": a_top, "first_downs": first_downs_allowed, "first_downs_allowed": first_downs, 
                   "pass_yards": pass_yards_allowed, "pass_yards_allowed": pass_yards, "penalty_yards": a_penalty_yards, 
                   "plays": a_plays, "rush_yards": rush_yards_allowed, "rush_yards_allowed": rush_yards, "sacks": sacked,
                   "sacked": sacks, "takeaways": turnovers, "turnovers": takeaways, "total_yards": total_yards_allowed, 
                   "total_yards_allowed": total_yards}

stats_dicts = [home_dictionary, away_dictionary]
nfl = pd.DataFrame(stats_dicts)

In [4]:
margin_ari_score = 0
margin_ari_opp = 0
margins = []
for key, row in nfl.iterrows():
    if row.team == "ARI":
        team_coeff = margin_ari_score
    else:
        res_team = "team[T." + row.team + "]"
        team_coeff = margin_res.params[res_team]
    if row.opp == "ARI":
        opp_coeff = margin_ari_opp
    else:
        res_opp = "team[T." + row.opp + "]"
        opp_coeff = margin_res.params[res_opp]
    if row.ha == "away":
        ha_coeff = margin_res.params["ha[T.home]"]*-1
    else:
        ha_coeff = margin_res.params["ha[T.home]"]*1
    margin_predict = margin_res.params.Intercept + margin_res.params.third_per*row['third_per'] + margin_res.params.third_per_allowed*row['third_per_allowed'] + margin_res.params.TOP*row['TOP'] + margin_res.params.first_downs*row['first_downs'] + margin_res.params.first_downs_allowed*row['first_downs_allowed'] + margin_res.params.pass_yards*row['pass_yards'] + margin_res.params.pass_yards_allowed*row['pass_yards_allowed'] + margin_res.params.penalty_yards*row['penalty_yards'] + margin_res.params.plays*row['plays'] + margin_res.params.rush_yards*row['rush_yards'] + margin_res.params.rush_yards_allowed*row['rush_yards_allowed'] + margin_res.params.sacked*row['sacked'] + margin_res.params.sacks*row['sacks'] + margin_res.params.takeaways*row['takeaways'] + margin_res.params.total_yards*row['total_yards'] + margin_res.params.total_yards_allowed*row['total_yards_allowed'] + margin_res.params.turnovers*row['turnovers'] + ha_coeff + team_coeff + opp_coeff
    margins.append(margin_predict)
away_margin = margins[0] + margins[1]
home_margin = -1*away_margin
pred_margins = [home_margin, away_margin]
print (pred_margins)

[17.852450441818608, -17.852450441818608]


In [5]:
total_ari_score = 0
total_ari_opp = 0
totals = []
for key, row in nfl.iterrows():
    if row.team == "ARI":
        team_coeff = total_ari_score
    else:
        res_team = "team[T." + row.team + "]"
        team_coeff = total_res.params[res_team]
    if row.opp == "ARI":
        opp_coeff = total_ari_opp
    else:
        res_opp = "team[T." + row.opp + "]"
        opp_coeff = total_res.params[res_opp]
    if row.ha == "away":
        ha_coeff = total_res.params["ha[T.home]"]*-1
    else:
        ha_coeff = total_res.params["ha[T.home]"]*1
    total_predict = total_res.params.Intercept + total_res.params.third_per*row['third_per'] + total_res.params.third_per_allowed*row['third_per_allowed'] + total_res.params.TOP*row['TOP'] + total_res.params.first_downs*row['first_downs'] + total_res.params.first_downs_allowed*row['first_downs_allowed'] + total_res.params.pass_yards*row['pass_yards'] + total_res.params.pass_yards_allowed*row['pass_yards_allowed'] + total_res.params.penalty_yards*row['penalty_yards'] + total_res.params.plays*row['plays'] + total_res.params.rush_yards*row['rush_yards'] + total_res.params.rush_yards_allowed*row['rush_yards_allowed'] + total_res.params.sacked*row['sacked'] + total_res.params.sacks*row['sacks'] + total_res.params.takeaways*row['takeaways'] + total_res.params.total_yards*row['total_yards'] + total_res.params.total_yards_allowed*row['total_yards_allowed'] + total_res.params.turnovers*row['turnovers'] + ha_coeff + team_coeff + opp_coeff
    totals.append(total_predict)
total_predicted = (totals[0] + totals[1])/2
pred_totals = [total_predicted, total_predicted]
print (pred_totals)

[59.219889556170315, 59.219889556170315]


In [6]:
pred_pfs = []
pred_pas = []
for x in np.arange(len(totals)):
    pred_margin = pred_margins[x]
    pred_total = pred_totals[x]
    a = np.array([[1,1], [1,-1]])
    b = np.array([[pred_total], [pred_margin]])
    points = np.linalg.solve(a,b)
    pf = (points[0][0])
    pa = (points[1][0])
    pred_pfs.append(pf)
    pred_pas.append(pa)
print (pred_pfs)
print (pred_pas)

[38.53616999899446, 20.683719557175856]
[20.683719557175856, 38.53616999899446]


In [7]:
print(f'{opponent} {round(pred_pas[0], 2)} @ {team} {round(pred_pfs[0], 2)}')

ARI 20.68 @ GBP 38.54
